Here, the process of the creation of operator matrices will be simplified, by not creating individual matrices for the T, V and then defining the operators as per the analytical expression.

Instead, we go the route of directly simplifying the expressions first into a condensed analytical matrix, and then defining the final operator matrix here directly.

In [1]:
import numpy as np 
from matplotlib import pyplot as plt 

# global parameters, as defined in the problem
xb = 20
Vb = 50
x0 = -10
k0 = 4
sigma0 = 2
C = 1/np.sqrt(sigma0*np.sqrt(np.pi))

# defining the spatial grid here
xs = -30
xe = 30
dx = 0.01
x = np.arange(xs, xe + dx, dx) 

# defining the temporal grid here
dt = 0.01
t = np.arange(0, 15 + dt, dt)



def Vx(x): # define potential function, with a step barrier at xb = 20 of height 50
    pot = np.zeros(len(x))
    for i in range(0, len(x)):
        if x[i]<=xb:
            pot[i]=0
        else:
            pot[i]=Vb
    return pot

def thomas(a, b, c, d, Nx):
    """Implementation of the Thomas algorithm. Implementation has been kept as general as possible, with certain modifications due to nature of variables used. However, the general structure is rigorously similar to the lecture notes for correspondence.

    Args:
        a (_complex_): Subdiagonal array, here supplanted with a constant as all values are same
        b (_ndarray.complex_): main-diagonal elements in an array
        c (_complex_): super-diagonal elements, used as a constant as all are same here
        d (_ndarray.complex_): contains the array D in the equation AX = D, where X is being solved for.
        Nx (_type_): length of the spatial grid.
    """
    
    b1 = np.zeros(Nx, dtype=complex) # new elements due to the thomas forward sweep
    d1 = np.zeros(Nx, dtype=complex)
    
    b1[0] = b[0] # as in the lecture notes, the first elements of the sweep are unchanged
    d1[0] = d[0] 
    for i in range(1, Nx): # sweep goes from 2nd element until last, first elements above
        b1[i] = b[i] - a / b1[i-1] * c 
        d1[i] = d[i] - a / b1[i-1] * d1[i-1]
        
    # the sweep allows for the construction of new matrices A1.X = D1, where we can now implement the final solution part.
    # as described in the lecture notes, we solve this from the last element using a reverse sweep
    
    x = np.zeros(Nx, dtype=complex) # NOT THE SPATIAL GRID, simply refers to the unknown being solved for
    x[Nx-1] = d[Nx-1] / b[Nx-1] # last element at index 6000 
    for i in range((Nx-1)-1, (0) -1, -1): # from index 5999 to 0 in reverse order
        x[i] = (d1[i] - c * x[i+1]) / b1[i]
        
    return x
     

In [2]:
Nt = len(t) # len(t): 1501
Nx = len(x) # len(x): 6001, required later to define the size of the operator arrays

# defining the potential array, as needed for creation of the operator matrices A+ and A-
V = Vx(x)


# creating the A+ operator matrix
# size of the spatial grid defines the size of the matrix here.
# since the matrix here only contains the diagonal, sub and super diagonal elements, only a single loop running though the diagonal is sufficient
A_plus = np.zeros((Nx, Nx), dtype=np.complex_)
for i in range(0, Nx - 1) :
    # leaving the last diagonal element out of this loop, as it does not have corresponding sub and super diagonal elements
    A_plus[i][i] = 1 - 1j*dt/2*(1/(dx**2) + V[i])
    
    A_plus[i][i+1] = 1j*dt/(4*(dx**2)) # sub and super same
    A_plus[i+1][i] = 1j*dt/(4*(dx**2))
A_plus[Nx-1][Nx-1] = 1 - 1j*dt/2*(1/(dx**2) + V[Nx-1]) # last diagonal element


# For the A- operator, we shall tackle it directly inside the THOMAS ALGORITHM IMPLEMENTATION
# essentially, the all the diagonal elements will be extracted into seperate arrays and then operated upon by the Thomas algo
# we create the three arrays a,b,c corresponding to the three diagonals in the A- operator matrix
# since a and c arrays have the same elements, we simply use constant values
# all three arrays hold complex values
b = np.zeros(Nx, dtype=np.complex_)
a = - 1j*dt/(4*(dx**2))
b = 1 + 1j*dt/2*(1/(dx**2) + V) 
c = - 1j*dt/(4*(dx**2))

# thomas algo defined in a function above


# Now that we have the propagation operators, we need to get the initial psi, as defined in the question
psi0 = np.zeros(len(x), dtype="complex") # as psi (x,t=0) here
for i in range(0, Nx):
    psi0[i] = C*np.exp(1j* k0 *x[i]) * np.exp( -((x[i]-x0)**2) / (2*sigma0**2) )
    
# plt.plot(x, np.real(psi0), label ="real")
# plt.plot(x, np.imag(psi0), label = "imag")
# plt.legend()


In [3]:
# we proceed to the implementation of the CN scheme
# scheme: A- * psi_n+1 = psi_n+0.5 = A+ * psi_n
# A general matrix holds the psi at each iteration to implement the time evolution at a later stage. Each row corresponds to psi at a given time step
psi = np.zeros((Nt, Nx), dtype=complex) # 1501 * 6001 matrix
psi[0] = psi0.copy()

psi_intermediate = np.zeros(Nx, dtype=complex) # 1501-size array, to hold the intermediate array in the CN step

for i in range(0, Nt-1): # goes from 0 to 1499 time-index
    psi_intermediate = A_plus @ psi[i]  # Step : psi_intmed = A+ * psi_n
    psi[i+1] = thomas(a,b,c, psi_intermediate, Nx) # Step : psi_n+1 = A-_inv * psi_intmed
    
# CN propagation successful !! 

# saving this matrix for further use in the other part
np.save("psi.npy", psi)
np.save("x.npy", x)
np.save("t.npy", t)

In [4]:
# plt.plot(x, psi[0], label = "start")
# plt.plot(x, psi[750], label = "end")
# plt.legend()

# FUNCTIONING VERIFIED USING PLOTS

In [5]:
from matplotlib.backends.backend_pdf import PdfPages

pdffile = "tdse.pdf"

with PdfPages(pdffile) as pdf:
    for i in range(0, len(t), 15):
        # creating plot
        plt.plot(x,psi[0], "r", label=f"at time step = {round(t[0],5)}")
        plt.plot(x,psi[i], "b",label=f"at time step = {round(t[i],5)}")
        plt.legend()
        plt.ylim(-0.5, 0.5)
        plt.grid()
        plt.xlabel("x")
        plt.ylabel("psi(x,t)")
        plt.title(f"Wavefunction: dx={dx}, iter = {len(t)-1}")
        plt.axvspan(20, 30, color = "grey", alpha = 0.3)
        
        # exporting to the pdf.
        pdf.savefig()
        plt.close()

/opt/homebrew/Caskroom/miniconda/base/envs/computational/lib/python3.12/site-packages/matplotlib/cbook.py:1762: ComplexWarning: Casting complex values to real discards the imaginary part
  return math.isfinite(val)
/opt/homebrew/Caskroom/miniconda/base/envs/computational/lib/python3.12/site-packages/matplotlib/cbook.py:1398: ComplexWarning: Casting complex values to real discards the imaginary part
  return np.asarray(x, float)
